# Main notebook

For quick and generic simulations.

## Imports

In [28]:
%matplotlib widget

# system
from src.system_creator import SystemCreator

# Grid
from src.utils import Grid, pos_in_grid, convert_to_grid_datatype

# Particles
from src.utils import Particle

# injection 
from src.utils import inject

# advection
from src.utils import advect
from src.utils import euler_explicit, leap_frog

# collisions
from src.utils import handler_wall_collision, handler_wall_collision_point, make_collisions_vectorized, make_collisions_out_walls, deal_with_corner

# utils 
from src.utils import gaussian, maxwellian_flux, maxwellian_mean_speed, get_mass_part, mean_free_path, mean_free_time

# systems
from src.utils import systems

# plotting 
from src.plotting import plot_boundaries, plot_particles, plot_grid, plot_system
from src.plotting import analysis

# collisions between particles
from src.utils import handler_particles_collisions, candidates # candidates, index_choosen_couples, probability, is_colliding, reflect, 

# saving 
from src.data import Saver

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
from pathlib import Path

# you can choose the seed here
np.random.seed(1111)

## System choice

Four default systems can be initialized using the following cell.

In [29]:
system_type = 'tube' # thruster, cylinder

# ---------------------- System --------------------
dz = 0.001
typical_lenght = 0.001 # typical size of the system (minimum distance between two walls for example)
                       # used for computing the mean free path later on (not used in the simulation)
                       # just useful to have an idea of it.

if system_type == 'tube':
    system, idx_out_walls, idx_in_wall = systems.system_rectangle(lx = 0.01, ly = 0.001)
    
elif system_type == 'square':
    system, idx_out_walls, idx_in_wall = systems.system_rectangle(lx = 0.01, ly = 0.01)

elif system_type == 'thruster':
    dp = 0.001
    system, idx_out_walls, idx_in_wall = systems.thruster_system(w_in = 5*dp, l_in = 3*dp, w1 = 3*dp, l1 = dp, l_int = dp, w2 = dp, l2 = 5*dp, w_out = 5*dp, l_out = dp, offsets = np.array([0,0]))

elif(system_type == 'cylinder'):
    system, idx_out_walls, idx_in_wall = systems.cylinder_system(res = 4, lx = 0.003, ly = 0.001, cx = 0.0015 , cy = 0.0005, r = 0.0001)

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

## DSMC grid creation

In [30]:
# grid :
mean_number_per_cell = 1000 # 200 is enough to have a "convergence", however, it is better to use more for better statistics (you could also average over more time steps in steady state)
max_number_per_cell = 10*mean_number_per_cell 
# factor 10 is completely overshot (Note : a future version will have dynamic arrays instead of static one)

if system_type == 'tube':
    resolutions = np.array((10,1), dtype = int) # tube

elif system_type == 'square':
    resolutions = np.array((3,3), dtype = int) # tube

elif system_type == 'thruster':
    resolutions = np.array((11,5), dtype = int)

elif(system_type == 'cylinder'):
    resolutions = np.array((9,9), dtype = int)
    
grid = Grid(resolutions, max_number_per_cell)

# --------- useful quantity for the simulation ------------ #
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]


## Particles

In [31]:
# ------------------ Particles Params ----------------- #

# particles density in the real system
density = 3.2e19 # m-3

part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10

temperature = 300 # K 
v_mean = maxwellian_mean_speed(temperature, mass)

size_array = 2*mean_number_per_cell*np.prod(resolutions) # max size for the array
container = Particle(part_type, charge, mass, radius, size_array)


# ------ useful quantities for the simulation (or for plotting) ----------- #
# ----------- based on params - should not be modified ---------------- #

# "mean number of particles in the simulated system"
n_simu = mean_number_per_cell*np.prod(resolutions)

# "mean number of particles in the real system"
n_real = volume_cell * density * np.prod(resolutions) 

# macro particules ratio = number of particles in the real system / number of macro part in the simulated system
mr = n_real/n_simu 

# density in the dsmc
density_dsmc = density/mr

# particle cross section (useful for particles collision)
cross_section = container.get_params()[3]  

# mean free path and time
mfp = mean_free_path(cross_section, density)
mft = mean_free_time(mfp, v_mean = v_mean) # min(typical_lenght,mfp)

## Initialization of particles

In [32]:
init_particles = False

if(init_particles):
    # you have to decice your strategy to initialize a 2D array of size Nx5
    # where N is the number of particle
    # and each particle is given [x, y, vx, vy, vz].
    init_size = mean_number_per_cell*np.prod(resolutions)
    extremal_values = system.get_extremal_values() 
    loc = 0
    vel_std = gaussian(temperature, mass)
    x = np.random.uniform(low = extremal_values['min_x'], high = extremal_values['max_x'], size = init_size)
    y = np.random.uniform(low = extremal_values['min_y'], high = extremal_values['max_y'], size = init_size)
    vx = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    vy = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    vz = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    new = np.stack((x,y,vx,vy,vz), axis = 1) 

    container.add_multiple(new)

## Injection params 

In [33]:
inject_particles = True

if(inject_particles):
    in_wall = segments[idx_in_wall]
    in_vect = np.array([a[idx_in_wall,1], -a[idx_in_wall,0]])
    
    # for the injection
    debit = maxwellian_flux(density_dsmc, v_mean)*np.linalg.norm(in_wall[:2]-in_wall[2:])*dz
    vel_std = gaussian(temperature, mass)


## Simulation parameters

In [34]:
# Simulation params
iterations = 1000
dt = 1e-6 # in sec.

# saving params 
saving_period = 10 # when do we save various data (see the simulation algo)
adding_period = 1 # when to we add to the dataframe that contains the particles position and velocity

# advection function - returns a 2D arrays containing the acceleration for each of the given particle
    # here arr is a N x 5 array. N particles, and for each one : x, y, vx, vy, vz is stored.
    # very simple for now - as there is no electric fields, nor any force in the system 
    # thus no acceleration
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))

# args is given to euler_explicit and then given to *f* (the advection function) in addition to arr and dt.
# in our case, it is not needed. However, we could imagine a system with an electric field computed at the setup phase, 
# and we would like to give it as an args.
args = []
scheme = euler_explicit

## Summing-up and plotting

In [39]:
print(f'Initializing a system of type {system_type}, of shape {system_shape} with {np.prod(resolutions)} cells.')
print(f'The number of particles (type {part_type}) in the system is {container.get_current()}.')
print(f'Mean free path : {mfp} m ; Mean free time : {mft} m')
print(f'In steady state, {n_simu} can be expect in the simulated system, which represents {round(n_real,3)} in the real system. The ratio between the two is {round(mr,3)}.')
print(f'The simulation lasts {iterations} iterations, with a time step of {dt} s. Simulation duration : {dt*iterations} s')
# Note:  HDF5 uses a different format than csv, and the size on the disk is much different that what is expected. Check out : https://support.hdfgroup.org/HDF5/doc/H5.intro.html
# Here, you can at least multiply by 4 the size (considering we save much more than )
print(f'Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save {n_simu} particles each time is {iterations//adding_period*n_simu*(5*4)/1024**2} MB.') 
plot_system(container.get_particles(), segments, radius, resolutions, system_shape, offsets);

Initializing a system of type tube, of shape [0.01  0.001] with 10 cells.
The number of particles (type I) in the system is 0.
In steady state, 10000 can be expect in the simulated system, which represents 320000000000.0 in the real system. The ratio between the two is 32000000.0.
The simulation lasts 1000 iterations, with a time step of 1e-06 s. Simulation duration : 0.001 s
Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save 10000 particles each time is 190.73486328125 MB.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Saving directory and name

In [ ]:
# which directory is used to save the data
# and under what name.
dir_path = Path('results/')
name = 'test_thruster.h5' 

saver = Saver(dir_path, name)

# Simulation

The next cell takes care of the simulation. It algo gives you an idea of the evolution of the number of particles in the system and of its very general state.

At the end, the *saver* which saves the data is closed and you can then analyse your simulation using *analysis.ipynb*.

In [ ]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# adding particle before the simulation - step 0
arr = container.get_particles()
df = df.append(pd.DataFrame(data=arr, index=[0]*arr.shape[0], columns = ['x','y','vx','vy','vz']))

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(1,iterations+1): # tqdm
    n1 = container.get_current()
                   
    # INJECTING PARTICLES
    new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    collisions_with_walls = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
        
        # the first one that is received is the number of particles colliding with walls.
        if(c == 1):
            collisions_with_walls = np.sum(count, where = count == True)
    
    idxes_out = np.concatenate(idxes_out)
    
    # TODO : make delete multiple better - currently the function creates a new array where as we can do it inplace.
    container.delete_multiple(idxes_out)
    
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
        # TODO : change the way it's done
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)
        
        # DSMC
        # TODO: make parallel (1st : note criticals functions in C++)
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # TODO: may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%adding_period == 0 or it == iterations-1):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
        
    if(it%saving_period == 0 or it == iterations-1): # saving if last iteration too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages, # evolution of the average number of particle per cell
                        'collisions_with_walls' : collisions_with_walls, # number of collisions with walls - evolution

                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz'])
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2, idxes_out.shape[0], c))
saver.close()